In [16]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

from lib import *
import subprocess
import psycopg2

In [17]:
# start_date_str = "2010-01-01T00:00:00"
# end_date_str = "2019-12-31T23:00:00"
# start_date = dt.datetime.strptime( start_date_str, '%Y-%m-%dT%H:%M:%S' )
# end_date = dt.datetime.strptime( end_date_str, '%Y-%m-%dT%H:%M:%S' )
# dates = pd.date_range(start_date, end_date, freq='h')

In [18]:
# input_path = "/media/lacie2022/data/meteo/ecmwf/era5/land/maps/"
# output_path = "/media/windows/projects/bias_correction/applications/era5/postprocessed/"
# mkNestedDir( output_path )

# variables = ['2t', 'tp']
             
# files = ['20102011.grib', '20122013.grib',
#          '20142015.grib', '20162017.grib',
#          '20182019.grib']

# # variables = ['tp']
             
# # files = ['20102011.grib']

In [19]:
def get_postgres_connection():

    db_name = 'meteo'
    db_user = 'postgres'
    db_password = 'password'
    db_host = '172.20.0.2'

    return psycopg2.connect(database=db_name, user=db_user, password=db_password, host=db_host)

In [20]:
# WITH dob_jobs_2014 AS (
#  SELECT
#  cartodb_id,
#  extract(epoch FROM (date_time - lag(date_time, 1) OVER(ORDER BY date_time))) AS duration_in_seconds
#  FROM tracking_eric
#  ORDER BY date_time
# )

# COPY dob_jobs_2014 to '/Users/chrislhenrick/development/nyc_dob_jobs/data/2014/dob_jobs_2014.csv' DELIMITER ',' CSV Header;

In [21]:
# conn = get_postgres_connection()
# try:
#     df = pd.read_sql('''
#         SELECT datetime, value, point
#         FROM ecmwf.era5_data
#         ORDER BY datetime;''',
#         # WHERE datetime = '2010-01-01 00:00';''',
#         conn);
# finally:
#     conn.close()

In [22]:
def get_postgres_engine():

    from sqlalchemy import create_engine

    db_name = 'meteo'
    db_user = 'postgres'
    db_password = 'password'
    db_host = '172.20.0.2'

    db_connection_url = '''postgresql://{user}:{password}@{host}:{port}/{db_name}'''.format(
        user = db_user,
        password = db_password,
        host = db_host,
        port = 5432,
        db_name = db_name
    )

    return create_engine(db_connection_url)

In [27]:
import geopandas as gpd
from geoalchemy2 import Geometry

for N in [1,2,5,6,7,8,9,10,12,13,14,15,17,18,19,20,21,22,23,24,25]:

    # read in the data
    gdf = gpd.read_file(f'/media/windows/projects/uncertainty/maps/SHP/mybasin_{N}.shp')

    # Drop nulls in the geometry column
    print('Dropping ' + str(gdf.geometry.isna().sum()) + ' nulls.')
    gdf = gdf.dropna(subset=['geometry'])

    for l in gdf.index:

        name = f'SB'+str(N).zfill(3)
        sql_insert = '''
            INSERT INTO "AA".basins(name, geom, area)
            VALUES ('{name}',  ST_Transform(ST_GeomFromText('{geom}',32632),4326), 
                ST_area(ST_GeomFromText('{geom}',32632)))
            ON CONFLICT DO NOTHING;'''.format(
                name = name,
                geom = gdf.loc[l,'geometry']
            )
        print(sql_insert)
        conn = get_postgres_connection()
        try:
            with conn.cursor() as cur:
                cur.execute(sql_insert)
                conn.commit()
        finally:
            conn.close()

Dropping 0 nulls.

            INSERT INTO "AA".basins(name, geom, area)
            VALUES ('SB001',  ST_Transform(ST_GeomFromText('POLYGON ((612873.75 5170148.75, 612873.75 5170173.75, 612898.75 5170173.75, 612898.75 5170198.75, 613048.75 5170198.75, 613048.75 5170223.75, 613198.75 5170223.75, 613198.75 5170248.75, 613298.75 5170248.75, 613298.75 5170273.75, 613373.75 5170273.75, 613373.75 5170298.75, 613448.75 5170298.75, 613448.75 5170323.75, 613498.75 5170323.75, 613498.75 5170348.75, 613523.75 5170348.75, 613523.75 5170373.75, 613573.75 5170373.75, 613573.75 5170398.75, 613598.75 5170398.75, 613598.75 5170423.75, 613648.75 5170423.75, 613648.75 5170448.75, 613698.75 5170448.75, 613698.75 5170473.75, 613723.75 5170473.75, 613723.75 5170498.75, 613748.75 5170498.75, 613748.75 5170523.75, 613773.75 5170523.75, 613773.75 5170548.75, 613948.75 5170548.75, 613948.75 5170573.75, 614123.75 5170573.75, 614123.75 5170598.75, 614223.75 5170598.75, 614223.75 5170698.75, 615223.75 5170698.75,

In [9]:
import geopandas as gpd
from geoalchemy2 import Geometry

# read in the data
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_0.shp')  ## ITALY
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_1.shp')  ## REGIONS
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_2.shp')  ## PROVINCES
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_3.shp')  ## MUNICIPALITIES
gdf = gpd.read_file('/media/lacie2022/data/meteo/ecmwf/era5/land/maps/AltoAdige_buffer_15km_4326.shp')

# Drop nulls in the geometry column
print('Dropping ' + str(gdf.geometry.isna().sum()) + ' nulls.')
gdf = gdf.dropna(subset=['geometry'])

for l in gdf.index:

    name = 'TAA_15km'
    sql_insert = '''INSERT INTO geometries.pranav(name, geom)
        VALUES ('{name}', ST_GeomFromText('{geom}',4326))
        ON CONFLICT DO NOTHING;'''.format(
            name = name,
            geom = gdf.loc[l,'geometry']
        )
    
    conn = get_postgres_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(sql_insert)
            conn.commit()
    finally:
        conn.close()

Dropping 0 nulls.


In [ ]:
import geopandas as gpd
from geoalchemy2 import Geometry

# read in the data
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_0.shp')  ## ITALY
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_1.shp')  ## REGIONS
gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_2.shp')  ## PROVINCES
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_3.shp')  ## MUNICIPALITIES

# Drop nulls in the geometry column
print('Dropping ' + str(gdf.geometry.isna().sum()) + ' nulls.')
gdf = gdf.dropna(subset=['geometry'])

for l in gdf.index:

    name = gdf.loc[l,'NAME_2']
    name = name.replace("'","`")

    region = gdf.loc[l,'NAME_1']
    region = region.replace("'","`")

    cc = gdf.loc[l,'CC_2']
    
    # print(cc)

    sql_insert = '''INSERT INTO geometries.eu_ita_provinces(region, name, geom, cc)
        VALUES ('{region}', '{name}', ST_GeomFromText('{geom}',4326), {cc})
        ON CONFLICT DO NOTHING;'''.format(
            region = region,
            name = name,
            geom = gdf.loc[l,'geometry'],
            cc = cc
        )

    # print(sql_insert)
    
    conn = get_postgres_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(sql_insert)
            conn.commit()
    finally:
        conn.close()

In [ ]:
gdf

In [ ]:
import geopandas as gpd
from geoalchemy2 import Geometry

# read in the data
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_0.shp')  ## ITALY
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_1.shp')  ## REGIONS
# gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_2.shp')  ## PROVINCES
gdf = gpd.read_file('/media/windows/data/maps/eu/it/ITA_adm/gadm36_ITA_3.shp')  ## MUNICIPALITIES

# Drop nulls in the geometry column
print('Dropping ' + str(gdf.geometry.isna().sum()) + ' nulls.')
gdf = gdf.dropna(subset=['geometry'])

for l in gdf.index:

    name = gdf.loc[l,'NAME_3']
    name = name.replace("'","`")

    province = gdf.loc[l,'NAME_2']
    province = province.replace("'","`")

    region = gdf.loc[l,'NAME_1']
    region = region.replace("'","`")

    sql_insert = '''INSERT INTO geometries.eu_ita_municipalites(region, province, name, geom)
        VALUES ('{region}','{province}','{name}', ST_GeomFromText('{geom}',4326))
        ON CONFLICT DO NOTHING;'''.format(
            region = region,
            province = province,
            name = name,
            geom = gdf.loc[l,'geometry']
        )

    # print(sql_insert)
    
    conn = get_postgres_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(sql_insert)
            conn.commit()
    finally:
        conn.close()

In [ ]:
gdf